# Raw Evaluation wrt GT

In [15]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d 

import pandas as pd
import os
import pprint

from evo.core.trajectory import PoseTrajectory3D
from evo.tools import plot
from evo.core import metrics
from evo.core import sync

%reload_ext autoreload
%autoreload 2

In [9]:
export_path = '/mnt/data/datasets/fgsp/gt/h_floor_anymal_1/'
n = 0

compslam_trajectories = []
gt_trajectories = []

for i in range(0, n+1):
    compslam_traj_filename = export_path + 'compslam.npy'
    gt_traj_filename = export_path + 'gt.npy'

    compslam_trajectories.append(np.load(compslam_traj_filename))
    gt_trajectories.append(np.load(gt_traj_filename))
assert len(gt_trajectories) == len(compslam_trajectories)


print(f'Loaded {len(gt_trajectories)} missions.')
print(f'GT poses: {gt_trajectories[0].shape[0]}')
print(f'Compslam poses: {compslam_trajectories[0].shape[0]}')

Loaded 1 missions.
GT poses: 2561
Compslam poses: 3230


In [23]:
def convert_to_traj(trajectory):
    ts = trajectory[:,0]
    xyz = trajectory[:,1:4]
    wxyz = trajectory[:,4:8]
    return PoseTrajectory3D(positions_xyz = xyz, orientations_quat_wxyz = wxyz, timestamps = ts)

idx = 0

compslam_traj = convert_to_traj(compslam_trajectories[idx])
gt_traj = convert_to_traj(gt_trajectories[idx])

max_diff = 0.01
gt_traj, compslam_traj = sync.associate_trajectories(gt_traj, compslam_traj, max_diff)
# compslam_traj.align(gt_traj, correct_scale=False, correct_only_scale=False)
gt_traj.align(compslam_traj, correct_scale=False, correct_only_scale=False)

(array([[ 3.97244369e-01, -9.17712684e-01,  5.83331653e-04],
        [ 9.17656728e-01,  3.97212956e-01, -1.13135790e-02],
        [ 1.01509081e-02,  5.02955378e-03,  9.99935829e-01]]),
 array([-2.33033979, -0.72228138,  0.95085114]),
 1.0)

In [24]:
delta = 5
delta_unit = metrics.Unit.frames
all_pairs = False
data = (gt_traj, compslam_traj)

print('--- Translational Part -----------------------------')
pose_relation = metrics.PoseRelation.translation_part
print('APE:')
ape_metric_trans = metrics.APE(pose_relation)
ape_metric_trans.process_data(data)
ape_stat_trans = ape_metric_trans.get_statistic(metrics.StatisticsType.rmse)
ape_stats_trans = ape_metric_trans.get_all_statistics()
pprint.pprint(ape_stats_trans)

print('RPE:')
rpe_metric_trans = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
rpe_metric_trans.process_data(data)
rpe_stat_trans = rpe_metric_trans.get_statistic(metrics.StatisticsType.rmse)
rpe_stats_trans = rpe_metric_trans.get_all_statistics()
pprint.pprint(rpe_stats_trans)
print('\n\n')

print('--- Rotational Part -----------------------------')
pose_relation = metrics.PoseRelation.rotation_angle_deg
print('APE:')
ape_metric_rot = metrics.APE(pose_relation)
ape_metric_rot.process_data(data)
ape_stat_rot = ape_metric_rot.get_statistic(metrics.StatisticsType.rmse)
ape_stats_rot = ape_metric_rot.get_all_statistics()
pprint.pprint(ape_stats_rot)

print('RPE:')
rpe_metric_rot = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
rpe_metric_rot.process_data(data)
rpe_stat_rot = rpe_metric_rot.get_statistic(metrics.StatisticsType.rmse)
rpe_stats_rot = rpe_metric_rot.get_all_statistics()
pprint.pprint(rpe_stats_rot)

def plot_evaluations_APE(ape_metric, ape_stats):
    # Trajectory plot
    fig = plt.figure(figsize=(8, 6), dpi=160)
    traj_by_label = {
        "estimate": compslam_traj,
        "reference": gt_traj
    }
    plot.trajectories(fig, traj_by_label, plot.PlotMode.xy)
    plt.show()

    # Trajectory plot
    seconds_from_start = [t - compslam_traj.timestamps[0] for t in compslam_traj.timestamps]
    fig = plt.figure(figsize=(8, 6), dpi=160)
    plot.error_array(fig.gca(), ape_metric.error, x_array=seconds_from_start,
                     statistics={s:v for s,v in ape_stats.items() if s != "sse"},
                     name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
    plt.show()

    plot_mode = plot.PlotMode.xy
    fig = plt.figure(figsize=(8, 6), dpi=160)
    ax = plot.prepare_axis(fig, plot_mode)
    plot.traj(ax, plot_mode, gt_traj, '--', "gray", "reference")
    plot.traj_colormap(ax, compslam_traj, ape_metric.error, 
                       plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
    ax.legend()
    plt.show()
    
plot_evaluations_APE(ape_metric_trans, ape_stats_trans)

--- Translational Part -----------------------------
APE:
{'max': 1.917590130321977,
 'mean': 0.23640019439710225,
 'median': 0.21460864411486913,
 'min': 0.0318689257021198,
 'rmse': 0.25400060143824854,
 'sse': 160.12947032792212,
 'std': 0.09290454036269845}
RPE:
{'max': 1.7417836852541873,
 'mean': 0.05591889698608506,
 'median': 0.0429839533782928,
 'min': 0.004581465890694848,
 'rmse': 0.10082997562949984,
 'sse': 5.042675256780984,
 'std': 0.08390328328084151}



--- Rotational Part -----------------------------
APE:
{'max': 90.1547445664446,
 'mean': 2.6426088322252834,
 'median': 2.3414594796437216,
 'min': 0.05916485423879902,
 'rmse': 3.481428435435587,
 'sse': 30082.693686529627,
 'std': 2.266486821250987}
RPE:
{'max': 89.70016455407675,
 'mean': 1.710818251334902,
 'median': 1.269009579072375,
 'min': 0.03357722585200684,
 'rmse': 4.5702358532161504,
 'sse': 10359.979653995088,
 'std': 4.237942503730053}


/usr/local/lib/python3.6/dist-packages/matplotlib/__init__.py:941: MatplotlibDeprecationWarning: nbagg.transparent is deprecated and ignored. Use figure.facecolor instead.
  mplDeprecation)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

In [29]:
compslam_trajectories[0][0:5,:]

array([[ 1.62823430e+18,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.62823430e+18,  5.64583628e-03,  3.93047646e-02,
         7.69039985e-02,  9.99999997e-01,  2.32033118e-05,
        -7.78890704e-05,  1.70213474e-05],
       [ 1.62823430e+18, -1.63190365e+00,  2.48728503e-01,
         6.07898273e-01,  7.11157954e-01,  8.41432222e-03,
         1.34434346e-02,  7.02853354e-01],
       [ 1.62823430e+18, -1.63259896e+00,  2.49769738e-01,
         6.10358779e-01,  7.10943427e-01,  7.52582317e-03,
         1.34268605e-02,  7.03080738e-01],
       [ 1.62823430e+18, -1.63022522e+00,  2.47914581e-01,
         6.16053243e-01,  7.11030414e-01,  8.99780459e-03,
         1.26060652e-02,  7.02990667e-01]])